In [36]:
from pinecone import Pinecone
from pinecone import ServerlessSpec
from langchain.text_splitter import RecursiveCharacterTextSplitter
import fitz
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings

In [1]:
import os
from dotenv import find_dotenv, load_dotenv
load_dotenv(find_dotenv("req.env"), override=True)

True

In [15]:
pc= Pinecone()

In [26]:
from pinecone import ServerlessSpec

index_name='sc2'
if index_name not in pc.list_indexes().names():
    pc.create_index(
    name=index_name,
    dimension=1536, # default dimension for text embedding, one of the recommended OpenAI's embedding models.
    metric='cosine', # algorithm to calculate distance between vectors.
    spec= ServerlessSpec(
        cloud='aws',
        region='us-east-1'
        )
    )
    print('Index is created successfully')

else:
    print(f'Index {index_name} already exists')


Index is created successfully


In [27]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
import fitz

with fitz.open("sc2.pdf") as pdf:
    # Extract text from each page
    churchill = ""
    for page in pdf:
        churchill += page.get_text()
    
text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
    length_function=len
)

In [28]:
print(f'now we have {len(chunks)} chunk')

now we have 2 chunk


In [29]:
# Initialize LLM and retriever
llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=1)
retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': 3})

# Create the RetrievalQA chain
chain = RetrievalQA.from_chain_type(llm=llm, chain_type='stuff', retriever=retriever)

In [31]:
# Query the chain
query = 'the flight is ready??'
answer = chain.run(query)
print(answer)

Based on the provided context, it appears that all the systems listed are ready, such as the helipad, battery voltage, EGI, radar, launcher, radome, FLIR, and DVR. Therefore, it could be inferred that the flight is indeed ready.


In [32]:
from langchain.schema import (
    SystemMessage,
    AIMessage,
    HumanMessage
)

messages = [
    SystemMessage(content="You are a checker for copilot. If all attributes are READY, you give the response like everything is ready. But if one of the attributes is NOT READY, you give the response like flight cannot be started."),
    HumanMessage(content="The flight is ready to fly?")
]

In [33]:
import os
from dotenv import find_dotenv, load_dotenv
load_dotenv(find_dotenv("req.env"), override=True)

True

In [34]:
pc= Pinecone()

In [35]:
index_name='sce2'
if index_name not in pc.list_indexes().names():
    pc.create_index(
    name=index_name,
    dimension=1536, # default dimension for text embedding, one of the recommended OpenAI's embedding models.
    metric='cosine', # algorithm to calculate distance between vectors.
    spec= ServerlessSpec(
        cloud='aws',
        region='us-east-1'
        )
    )
    print('Index is created successfully')

else:
    print(f'Index {index_name} already exists')


Index is created successfully


In [37]:
embedding = OpenAIEmbeddings(model='text-embedding-3-small')

In [40]:
vector_store = Pinecone.from_documents(chunks, embedding, index_name=index_name)

AttributeError: from_documents is not a top-level attribute of the Pinecone class provided by pinecone's official python package developed at https://github.com/pinecone-io/pinecone-python-client. You may have a name collision with an export from another dependency in your project that wraps Pinecone functionality and exports a similarly named class. Please refer to the following knowledge base article for more information: https://docs.pinecone.io/troubleshooting/pinecone-attribute-errors-with-langchain


In [ ]:
output= llm.invoke(messages)
print(output.content)